<a href="https://colab.research.google.com/github/MiguelCarbo/Anomalies-Detection-TFG/blob/main/PYOD_Anomalies_795802.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Reset Variables
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
# C:\Users\mivar\anaconda3\pkgs\pyod-1.0.7-pyhd8ed1ab_0\site-packages
# C:\Users\mivar\Documents\04_MIGUEL\01UNIVERSIDAD\TFG\Zeek\Packages\zat-main\zat-main

# Packet Unzipping
!unzip pyod.zip;
!unzip zat.zip;

# Import all models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD


# Local Imports
import zat;
from zat.log_to_dataframe import LogToDataFrame;
from zat.dataframe_to_matrix import DataFrameToMatrix;

# Packet Imports
import pandas as pd;
import numpy as np;
import sklearn;
from sklearn.ensemble import IsolationForest;
from sklearn.decomposition import PCA;
from sklearn.cluster import KMeans, DBSCAN;
from sklearn.preprocessing import LabelEncoder

from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager

# Version Printing
print('zat: {:s}'.format(zat.__version__))
print('Pandas: {:s}'.format(pd.__version__))
print('Numpy: {:s}'.format(np.__version__))
print('Scikit Learn Version:', sklearn.__version__)
# print('PYOD Version: {:s}'.format(pyod.__version__))

Archive:  pyod.zip
replace pyod/models/abod.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: pyod/models/abod.py     
  inflating: pyod/models/alad.py     
  inflating: pyod/models/anogan.py   
  inflating: pyod/models/auto_encoder.py  
  inflating: pyod/models/auto_encoder_torch.py  
  inflating: pyod/models/base.py     
  inflating: pyod/models/base_dl.py  
  inflating: pyod/models/cblof.py    
  inflating: pyod/models/cd.py       
  inflating: pyod/models/cof.py      
  inflating: pyod/models/combination.py  
  inflating: pyod/models/copod.py    
  inflating: pyod/models/deep_svdd.py  
  inflating: pyod/models/ecod.py     
  inflating: pyod/models/feature_bagging.py  
  inflating: pyod/models/gaal_base.py  
  inflating: pyod/models/gmm.py      
  inflating: pyod/models/hbos.py     
  inflating: pyod/models/iforest.py  
  inflating: pyod/models/inne.py     
  inflating: pyod/models/kde.py      
  inflating: pyod/models/knn.py      
  inflating: pyod/models/kpca.py     
  infl

### 1. LOG TO CREATE DATAFRAME FROM ###

In [ ]:
# Create a Pandas dataframe from the Zeek log
log_to_df = LogToDataFrame();
conn_df = log_to_df.create_dataframe('mpli_conn_ts.log');
print('Read in {:d} Rows...'.format(len(conn_df)))
conn_df.columns

Read in 323331 Rows...


Index(['sourceAddress', 'sourcePort', 'destinationAddress', 'destinationPort',
       'service', 'duration', 'orig_bytes', 'resp_bytes', 'history',
       'orig_pkts', 'resp_pkts', 'mediaOrigen', 'mediaResp', 'desvOrigen',
       'desvResp', 'mediaTime', 'desvTime'],
      dtype='object')

### 2. FEATURE SELECTION ###

#### 2.A CONN.LOG FEATURES ####

In [ ]:
# Pick some features that might be interesting
features = ['proto','service','duration','orig_bytes','resp_bytes','conn_state',
       'local_orig','local_resp','missed_bytes','history','orig_pkts',
       'resp_pkts', 'tunnel_parents'];

# features = ['service','duration','orig_bytes','resp_bytes','missed_bytes','history','tunnel_parents'];

#### 2.B AMPLI_CONN.LOG FEATURES ####

In [ ]:
# Lorena's Script Features

# Complete Features
full_features = ['ts','sourceAddress','sourcePort','destinationAddress','destinationPort','service','duration','orig_bytes','resp_bytes',
           'history','orig_pkts','resp_pkts','mediaOrigen','mediaResp','desvOrigen','desvResp','mediaTime','desvTime' ]

features = ['service','duration','orig_bytes','resp_bytes','history','orig_pkts','resp_pkts','mediaOrigen','mediaResp','desvOrigen','desvResp','mediaTime','desvTime' ]

### 3. DATA PREPARATION ###

#### 3.1 DATA CLEANSING ####

In [ ]:
# Feature Filtering + NaNs removal
conn_df = conn_df.dropna()
conn_features_df = conn_df[features]

In [ ]:
# Data Preprocessing (Removing NaNs)
for feature in features:
    if (conn_features_df[feature].dtype.name == 'category') and ('0' not in conn_features_df[feature].cat.categories):
        conn_features_df[feature] = conn_features_df[feature].cat.add_categories(['0'])
        conn_features_df[feature] = conn_features_df[feature].fillna('0')
    elif pd.api.types.is_timedelta64_dtype(conn_features_df[feature]): 
        conn_features_df[feature] = conn_features_df[feature].fillna(pd.Timedelta(0))
        conn_features_df[feature] = conn_features_df[feature].dt.total_seconds().astype(int)

    else: 
        conn_features_df[feature] = conn_features_df[feature].fillna(0)

<ipython-input-132-b17b2be7c65d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conn_features_df[feature] = conn_features_df[feature].cat.add_categories(['0'])
<ipython-input-132-b17b2be7c65d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conn_features_df[feature] = conn_features_df[feature].fillna('0')
<ipython-input-132-b17b2be7c65d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

#### 3.2 LABEL ENCODING ####

In [ ]:
# Relevant Considerations
# -- The ABOD (Angle-based Outlier Detection) algorithm is a distance-based outlier 
# -- detection algorithm that measures the angles between points in a high-dimensional space to detect outliers. 

# -- As a distance-based algorithm, it requires that all input features be numeric, as it relies on computing distances between points in the feature space.
# -- If your data contains non-numeric features, such as string values in some columns, then the ABOD algorithm will not be able to compute distances correctly 
# -- and will not be able to fit the data. This is normal behavior for distance-based algorithms like ABOD.

# -- To use the ABOD algorithm on your data, you will need to preprocess your data to convert the string features into numeric ones. 
# -- There are several ways to do this, depending on the specific characteristics of your data. One common approach is to use one-hot encoding or 
# -- label encoding to convert categorical features into numerical features that can be used by the algorithm.

# -- Alternatively, you may consider using a different outlier detection algorithm that can handle mixed data types, 
# -- such as the Isolation Forest algorithm or the Local Outlier Factor algorithm. These algorithms can handle both numeric and categorical features, 
# -- and can be used on mixed data types without requiring feature preprocessing.

In [ ]:
# Data Preprocessing - LabelEncoding
# -- Another option might be using OneHotEncoding
le = LabelEncoder()
for feature in features:
    if (conn_features_df[feature].dtype.name == 'category') :
        print(feature)
        conn_features_df[feature] = le.fit_transform(conn_features_df[feature])

# Dataframe conversion to float
conn_features_df = conn_features_df.astype(float)


service
history


#### 3.3 NUMPY ARRAY CREATION ####

In [ ]:
# Convert the dataframe to a numpy array
X = conn_features_df.to_numpy()
print(X[0:10,:])
print(X.shape)

[[1.40000000e+01 0.00000000e+00 2.20000000e+02 1.79000000e+02
  4.75000000e+02 8.00000000e+00 8.00000000e+00 1.37500000e+01
  1.11875000e+01 5.32535210e+01 4.33290010e+01 4.40000000e-05
  1.06000000e-04]
 [1.40000000e+01 0.00000000e+00 3.00000000e+02 1.47200000e+03
  4.37000000e+02 1.00000000e+01 6.00000000e+00 1.66666670e+01
  8.17777780e+01 6.87184270e+01 3.37178416e+02 5.40000000e-05
  1.66000000e-04]
 [1.40000000e+01 0.00000000e+00 1.38800000e+03 1.41500000e+03
  4.75000000e+02 8.00000000e+00 8.00000000e+00 8.67500000e+01
  8.84375000e+01 3.35981305e+02 3.42516965e+02 7.60000000e-05
  1.88000000e-04]
 [1.40000000e+01 0.00000000e+00 2.26000000e+02 1.85000000e+02
  4.75000000e+02 8.00000000e+00 8.00000000e+00 1.41250000e+01
  1.15625000e+01 5.47058900e+01 4.47813700e+01 3.50000000e-05
  7.20000000e-05]
 [1.40000000e+01 0.00000000e+00 3.06000000e+02 1.47200000e+03
  4.37000000e+02 1.00000000e+01 6.00000000e+00 1.70000000e+01
  8.17777780e+01 7.00927960e+01 3.37178416e+02 4.70000000e-0

### 4. CLASSIFICATION ALGORITHMS ###

In [ ]:
# Random State 42
# -- In NumPy, the random_state parameter is used to set the seed for the 
# -- random number generator, which determines the sequence of random numbers 
# -- generated by NumPy's random functions.

# -- When you set random_state to a fixed value, you get a reproducible sequence 
# -- of random numbers every time you run your code. 
# -- This can be useful when you need to compare results across multiple runs 
# -- or when you want to ensure that the random numbers are the same each time you run your code.

# -- In the case of random_state=42, the number 42 is an arbitrary value that 
# -- is commonly used as a default seed in many programming examples and tutorials.

rs = np.random.RandomState(42)

# Outlier detectetion algorithms
clf = {
    # 'Angle-based Outlier Detector (ABOD)': ABOD(contamination=0.1),
    # 'Cluster-based Local Outlier Factor (CBLOF)': CBLOF(contamination=0.1, check_estimator=False, random_state=rs),
    'Isolation Forest': IForest(contamination=0.1, random_state=rs),
    # 'K Nearest Neighbors (KNN)': KNN(contamination=0.1),
    # 'Average KNN': KNN(method='mean', contamination=0.1),
    # 'Local Outlier Factor (LOF)': LOF(n_neighbors=35, contamination=0.1),
    'Copula Based Outlier Detector (COPOD)': COPOD(contamination=0.1),
    'Emipirical-Cumulative-Distribution-based Outlier Detector (ECOD)': ECOD(contamination=0.1),
    # 'One-class SVM (OCSVM)': OCSVM(contamination=0.1),
}

In [ ]:
# Show all detectors
for i, classifier in enumerate(clf.keys()):
    print('Model', i + 1, classifier)

Model 1 Isolation Forest
Model 2 Copula Based Outlier Detector (COPOD)
Model 3 Emipirical-Cumulative-Distribution-based Outlier Detector (ECOD)


### 5. MODEL FITTING - TRAINING - PREDICTION ###

In [ ]:
clusters_separation = [0]
clf_names = list(clf.keys())
outliers_pred = np.zeros((X.shape[0], len(clf_names)))

# Fit the models with the generated data and compare model performances
for i, offset in enumerate(clusters_separation):
    np.random.seed(42)

    # Fit the model
    for i, (classifier_name, classifier) in enumerate(clf.items()):

        print(i + 1, 'fitting', classifier_name)

        # Fit the data and tag outliers
        classifier.fit(X)

        # Scores prediction
        scores_pred = classifier.decision_function(X) * -1
        
        # Outliers detection (based on prediction)
        y_pred = classifier.predict(X)
        outliers_pred[:,i] = y_pred;
        
        # Threshold explanation
        # -- The command threshold = percentile(scores_pred, 100 * 0.1) is used 
        # -- to set a threshold for outlier detection based on the scores predicted by the algorithm.

        # -- Scores_pred is a one-dimensional NumPy array of scores predicted by the outlier detection algorithm. 
        # -- The scores represent how "outlying" each data point is, with higher scores indicating a higher likelihood of being an outlier.

        # -- Percentile(scores_pred, 100 * 0.1) computes the 10th percentile of the scores_pred array, which is the score 
        # -- value below which 10% of the data points fall. In other words, if a data point has a score below this threshold, it is considered an outlier.

        # --  The resulting threshold value is assigned to the threshold variable for later use.The specific value of 0.1 in the command percentile(scores_pred, 100 * 0.1) is 
        # -- arbitrary and can be adjusted depending on the specific problem and the desired level of sensitivity to outliers. 

        # threshold = percentile(scores_pred, 100 * 0.05)
        # num_outliers = (y_pred > threshold).sum()

1 fitting Isolation Forest
2 fitting Copula Based Outlier Detector (COPOD)
3 fitting Emipirical-Cumulative-Distribution-based Outlier Detector (ECOD)


### 6. RESULT ANALYSIS ###

### 6.1 OUTLIERS DATAFRAMES ###

In [ ]:
# New Dataframes to store each Outlier
ABOD_out_df = pd.DataFrame()
CBLOF_out_df = pd.DataFrame()
IFOREST_out_df = pd.DataFrame()
KNN_out_df = pd.DataFrame()
AKNN_out_df = pd.DataFrame()
LOF_out_df = pd.DataFrame()
ECOD_out_df = pd.DataFrame()
COPOD_out_df = pd.DataFrame()

# Dropping NaNs from conn_df
conn_df = log_to_df.create_dataframe('mpli_conn_ts.log');
conn_df = conn_df.dropna()

# Outliers Stored for each algorithm
for i in range(outliers_pred.shape[0]):
    if outliers_pred[i,0] == 1:
        IFOREST_out_df = IFOREST_out_df.append(conn_df.iloc[i])
    if outliers_pred[i,1] == 1:
        COPOD_out_df = COPOD_out_df.append(conn_df.iloc[i])
    if outliers_pred[i,2] == 1:
        ECOD_out_df = ECOD_out_df.append(conn_df.iloc[i])

# Numeric Data Types Correction
IFOREST_out_df['sourcePort'] = np.uint16(IFOREST_out_df['sourcePort'])
IFOREST_out_df['destinationPort'] = np.uint16(IFOREST_out_df['destinationPort'])
COPOD_out_df['sourcePort'] = np.uint16(COPOD_out_df['sourcePort'])
COPOD_out_df['destinationPort'] = np.uint16(COPOD_out_df['destinationPort'])
ECOD_out_df['sourcePort'] = np.uint16(ECOD_out_df['sourcePort'])
ECOD_out_df['destinationPort'] = np.uint16(ECOD_out_df['destinationPort'])


#    if outliers_pred[i, x] == 1:
#        ABOD_out_df = ABOD_out_df.append(conn_df.iloc[i])
#    if outliers_pred[i, x] == 1:
#        CBLOF_out_df = CBLOF_out_df.append(conn_df.iloc[i])
#    if outliers_pred[i, x] == 1:
#        IFOREST_out_df = IFOREST_out_df.append(conn_df.iloc[i])
#    if outliers_pred[i, x] == 1:
#        KNN_out_df = KNN_out_df.append(conn_df.iloc[i])
#    if outliers_pred[i, x] == 1:
#        AKNN_out_df = AKNN_out_df.append(conn_df.iloc[i])
#    if outliers_pred[i, x] == 1:
#        LOF_out_df = LOF_out_df.append(conn_df.iloc[i])

### 6.2 ANOMALOUS IPs EXTRACTION ###

In [ ]:
# We extract unique malign FlowID Values
IFOREST_out_df['flux_id'] = IFOREST_out_df['sourceAddress'].astype(str) + IFOREST_out_df['destinationAddress'].astype(str) + IFOREST_out_df['sourcePort'].astype(str) + IFOREST_out_df['destinationPort'].astype(str)
IFOREST_flow_id_values = IFOREST_out_df['flux_id'].unique()

COPOD_out_df['flux_id'] = COPOD_out_df['sourceAddress'].astype(str) + COPOD_out_df['destinationAddress'].astype(str) + COPOD_out_df['sourcePort'].astype(str) + COPOD_out_df['destinationPort'].astype(str)
COPOD_flow_id_values = COPOD_out_df['flux_id'].unique()

ECOD_out_df['flux_id'] = ECOD_out_df['sourceAddress'].astype(str) + ECOD_out_df['destinationAddress'].astype(str) + ECOD_out_df['sourcePort'].astype(str) + ECOD_out_df['destinationPort'].astype(str)
ECOD_flow_id_values = ECOD_out_df['flux_id'].unique()


### 7. RESULT ANALYSIS ###

#### 7.A FROM ORIGINAL CSV ####

In [ ]:
# Original Source with Labeled Data

original_df = pd.read_csv('Tuesday-WorkingHours.pcap_ISCX.csv')
print(original_df.columns)

# Drop all rows in which column 'feature' does not fit a criteria
malign_original_df = original_df.drop(original_df[original_df[' Label'] == 'BENIGN'].index)

# Connection = SrcIP + DstIP
if (' Source IP' and ' Destination IP' and ' Source Port' and ' Destination Port') in original_df.columns:
    malign_original_df = malign_original_df.rename(columns={' Source IP': 'sourceAddress'})
    malign_original_df = malign_original_df.rename(columns={' Destination IP': 'destinationAddress'})
    malign_original_df = malign_original_df.rename(columns={' Source Port': 'sourcePort'})
    malign_original_df = malign_original_df.rename(columns={' Destination Port': 'destinationPort'})

# Unique malign labeled Flows
malign_original_df['flux_id'] = malign_original_df['sourceAddress'].astype(str) + malign_original_df['destinationAddress'].astype(str) + malign_original_df['sourcePort'].astype(str) + malign_original_df['destinationPort'].astype(str)
malign_flux_id_values = malign_original_df['flux_id'].unique()


In [ ]:
# Positive Detection
detected_positives = 0

for i in range (0, len(clf_names)):
    if i == 0:
        for flow_id in IFOREST_flow_id_values:
            # Check if the concatenated value is in the list of ip_src_dest values
            if flow_id in malign_flux_id_values:
                detected_positives+=1
        print('- IFOREST Results -')
        print('Detected Positives:', detected_positives)
        print('False Positives: ', len(IFOREST_flow_id_values) - detected_positives)
        print('Undetected Malign:', len(malign_flux_id_values) - detected_positives)
    if i == 1:
        for flow_id in COPOD_flow_id_values:
            # Check if the concatenated value is in the list of ip_src_dest values
            if flow_id in malign_flux_id_values:
                detected_positives+=1
        print('- COPOD Results -')
        print('Detected Positives:', detected_positives)
        print('False Positives: ', len(COPOD_flow_id_values) - detected_positives)
        print('Undetected Malign:', len(malign_flux_id_values) - detected_positives)
    if i == 2:
        for flow_id in ECOD_flow_id_values:
            # Check if the concatenated value is in the list of ip_src_dest values
            if flow_id in malign_flux_id_values:
                detected_positives+=1
        print('- ECOD Results -')
        print('Detected Positives:', detected_positives)
        print('False Positives: ', len(ECOD_flow_id_values) - detected_positives)
        print('Undetected Malign:', len(malign_flux_id_values) - detected_positives)


- IFOREST Results -
Detected Positives: 4055
False Positives:  26230
Undetected Malign: 2915
- COPOD Results -
Detected Positives: 4100
False Positives:  26129
Undetected Malign: 2870
- ECOD Results -
Detected Positives: 8101
False Positives:  21110
Undetected Malign: -1131


#### 7.B FROM SCRIPT LABELED .LOG FILE ####

In [ ]:
# Dataframe etiquetado según el Script creado
labeled_conn_df = log_to_df.create_dataframe('mpli_conn_label.log');
malign_conn_df = labeled_conn_df[labeled_conn_df['label'] == 'MALIGN'];
malign_conn_df['flux_id'] = malign_conn_df['sourceAddress'].astype(str) + malign_conn_df['destinationAddress'].astype(str) + malign_conn_df['sourcePort'].astype(str) + malign_conn_df['destinationPort'].astype(str)

<ipython-input-168-74832633f230>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malign_conn_df['flux_id'] = malign_conn_df['sourceAddress'].astype(str) + malign_conn_df['destinationAddress'].astype(str) + malign_conn_df['sourcePort'].astype(str) + malign_conn_df['destinationPort'].astype(str)


In [ ]:
# Detectamos Detecciones Exitosas -> INNER JOIN

# IFOREST RESULTS
IFOREST_malign_common_df_1 = pd.merge(IFOREST_out_df, malign_conn_df, on='flux_id')

print('- IFOREST Results - ')
positives1 = len(IFOREST_malign_common_df_1)
print('Detected positives: ' +str(positives1))
print('Total Malign: ' + str(malign_conn_df.shape[0]))
print('Undetected malign: ' +str(malign_conn_df.shape[0] - positives1))
print('')

# COPOD RESULTS
COPOD_malign_common_df_1 = pd.merge(COPOD_out_df, malign_conn_df, on='flux_id')

print('- COPOD Results - ')
positives1 = len(COPOD_malign_common_df_1)
print(positives1)

# ECOD RESULTS
ECOD_malign_common_df_1 = pd.merge(ECOD_out_df, malign_conn_df, on='flux_id')

print('- ECOD Results - ')
positives1 = len(ECOD_malign_common_df_1)
print(positives1)


- IFOREST Results - 
Detected positives: 3975
Total Malign: 6401
Undetected malign: 2426
- COPOD Results - 
24
- ECOD Results - 
3937
